In [7]:
from dbutil import conn
import pyTigerGraph as tg 

In [8]:
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
Edge Types:

Graphs:
Jobs:


JSON API version: v2
Syntax version: v2



AttributeError: module 'pyTigerGraph' has no attribute 'Gsql'

In [14]:
# Type, address, city, state, zip, price, beds, baths, location, square feet, lot size, year built, days on market, $/square feet, hoa/month, latitude, longitude
print(conn.gsql('''
create vertex property (primary_id mls_id string, address string, city string, state string, zip string, price float, lat double, lng double, beds float, baths float, sqft float, url string, type string)
                      
create undirected edge comparable(from property, to property, reason string, created_at datetime)

create vertex trait(primary_id name string)

create undirected edge has_trait(from property, to trait)

create graph propgraph(property, comparable, trait, has_trait)
''', options=[]))

Semantic Check Fails: The vertex name property is used by another object! Please use a different name.
Failed to create vertex types: [property].


In [13]:
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX property(PRIMARY_ID mls_id STRING, address STRING, city STRING, state STRING, zip STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types:
- UNDIRECTED EDGE comparable(FROM property, TO property, reason STRING, created_at DATETIME)

Graphs:
- Graph propgraph(property:v, comparable:e)
Jobs:


JSON API version: v2
Syntax version: v2

